In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import missingno as msn
import dexplot as dxp
import numpy as np
from scipy.stats import spearmanr
from datetime import timedelta

In [2]:

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb

In [3]:

train=pd.read_csv("../training_set_VU_DM.csv")

In [4]:
## dropping columns with high nulls

to_drop= ['comp1_rate',
'comp1_inv',
'comp1_rate_percent_diff',
'comp2_rate',
'comp2_inv',
'comp2_rate_percent_diff',
'comp3_rate',
'comp3_inv',
'comp3_rate_percent_diff',
'comp4_rate',
'comp4_inv',
'comp4_rate_percent_diff',
'comp5_rate',
'comp5_inv',
'comp5_rate_percent_diff',
'comp6_rate',
'comp6_inv',
'comp6_rate_percent_diff',
'comp7_rate',
'comp7_inv',
'comp7_rate_percent_diff',
'comp8_rate',
'comp8_inv',
'comp8_rate_percent_diff']

#creating dependant variable 

conditions = [
    (train['click_bool']> 0)
]

choices = [1]
train['outcome'] = np.select(conditions, choices, default=0)



train = train.drop(to_drop,  axis=1)#.sample(frac=0.2, replace=True, random_state=1)

#filtering out price outliers 
train = train[train.price_usd < 3001]

In [5]:
train

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,click_bool,gross_bookings_usd,booking_bool,outcome
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,0,1,1,NaN,NaN,1,0,NaN,0,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,0,1,1,NaN,NaN,1,0,NaN,0,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,0,1,1,NaN,NaN,1,0,NaN,0,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,0,1,1,NaN,NaN,1,0,NaN,0,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,0,1,1,NaN,NaN,1,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,77700,3,4.0,...,0,1,0,NaN,550.92,0,0,NaN,0,0
4958343,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,88083,3,4.0,...,0,1,0,NaN,553.14,0,0,NaN,0,0
4958344,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,94508,3,3.5,...,0,1,0,NaN,544.43,0,0,NaN,0,0
4958345,332785,2013-06-30 19:55:18,5,219,NaN,NaN,219,128360,3,5.0,...,0,1,0,NaN,550.38,0,1,157.84,1,1


In [6]:
# This function assigns grade 5 if the hotel is booked, grade 1 if it is only clicked, and 0 otherwise.

def get_grade(row):
    if row['booking_bool'] == 1:
        return 5
    else:
        if row['click_bool'] == 1:
            return 1
        else:
            return 0
        
        
train['grade'] = train.apply(lambda row: get_grade(row), axis=1)

In [7]:
train.head() # 32 columns

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,click_bool,gross_bookings_usd,booking_bool,outcome,grade
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,1,1,NaN,NaN,1,0,NaN,0,0,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,1,1,NaN,NaN,1,0,NaN,0,0,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,1,1,NaN,NaN,1,0,NaN,0,0,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,1,1,NaN,NaN,1,0,NaN,0,0,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,1,1,NaN,NaN,1,0,NaN,0,0,0


In [ ]:
# train['id_with_booking'] = train.groupby('srch_id')['booking_bool'].transform('sum')
# print(np.unique(train['id_with_booking']))

# train2 = train.drop(train[(train['id_with_booking'] == 0)].index)
# train2 = train2.drop(['id_with_booking'],  axis=1)#.sample(frac=0.2, replace=True, random_state=1)

# train = train2

In [8]:
## creating time related features
    
train['date_time'] = pd.to_datetime(train['date_time'], errors='coerce')
train["weekday"] = train["date_time"].dt.weekday
train["week_of_year"] = train["date_time"].dt.week
train["month"] = train["date_time"].dt.month
train["hour"] = train["date_time"].dt.hour
## total time elapsed - allows model to learn continous trend over time to a degree
train["time_epoch"] = train["date_time"].astype('int64')//1e9
train.loc[ train['hour'] < 6, 'day_time'] = 1
train.loc[(train['hour'] >=6) & (train['hour'] <= 11), 'day_time'] = 2
train.loc[(train['hour'] >= 12) & (train['hour'] <= 17), 'day_time'] = 3
train.loc[(train['hour'] > 18) , 'day_time'] = 4

<ipython-input-8-8a4ad4be83d5>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train["week_of_year"] = train["date_time"].dt.week


In [9]:
#more time related features

train['srch_for_date'] = train['date_time']  + pd.to_timedelta(train['srch_booking_window'], unit='D')
train["srch_for_month"] = train["srch_for_date"].dt.month
train.loc[ train['srch_for_month'] <= 3, 'seasonality'] = 1
train.loc[(train['srch_for_month'] > 3) & (train['srch_for_month'] <= 6), 'seasonality'] = 2
train.loc[(train['srch_for_month'] > 6) & (train['srch_for_month'] <= 9), 'seasonality'] = 3
train.loc[(train['srch_for_month'] > 9) , 'seasonality'] = 4

train.loc[ train['month'] <= 3, 'qrtr'] = 1
train.loc[(train['month'] > 3) & (train['month'] <= 6), 'qrtr'] = 2
train.loc[(train['month'] > 6) & (train['month'] <= 9), 'qrtr'] = 3
train.loc[(train['month'] > 9) , 'qrtr'] = 4

In [10]:
#filling in missing values for prop_review_score and prop_starrating using average rating 
#based off similar priced hotels 
train['price_bin'] = pd.qcut(train['price_usd'], q=80, precision=0)
train['mean_price_bin_star'] = train.groupby('price_bin')['prop_starrating'].transform('mean')
train['mean_price_bin_review'] = train.groupby('price_bin')['prop_review_score'].transform('mean')
train['prop_review_score'] = train['prop_review_score'].fillna(0)
train.loc[train.prop_review_score == 0, "prop_review_score"] = train.mean_price_bin_review
train.loc[train.prop_starrating == 0, "prop_starrating"] = train.mean_price_bin_star

In [11]:
#comparing price for hotels with other hotels in the same search

train['max_price'] = train.groupby('srch_id')['price_usd'].transform('max') 
train['min_price'] = train.groupby('srch_id')['price_usd'].transform('min') 
train['mean_price'] = train.groupby('srch_id')['price_usd'].transform('mean') 
train['diff_max_price'] =(train['max_price'] -train['price_usd']) / train['price_usd']
train['diff_min_price'] = (train['min_price'] - train['price_usd'])/ train['price_usd']
train['diff_min_price'] = train['diff_min_price'].replace(np.nan, 0)
train['diff_mean_price'] = (train['mean_price'] - train['price_usd']) / train['price_usd']
train['price_review_value'] =  train['prop_review_score'] /train['mean_price_bin_review'] 


In [12]:
# getting actual click and booking statistics for each hotel & if there was a booking - how many adults,children,rooms

train['hotel_show_prob'] =  train.groupby('prop_id')['prop_id'].transform('count') / (train.srch_id.unique()).size
train_calcs = train[train.booking_bool == 1]
train_calcs['hotel_avg_children'] = train_calcs.groupby('prop_id')['srch_children_count'].transform('mean') 
train_calcs['hotel_avg_adult'] = train_calcs.groupby('prop_id')['srch_adults_count'].transform('mean') 
train_calcs['hotel_avg_stay'] = train_calcs.groupby('prop_id')['srch_length_of_stay'].transform('mean') 
train_calcs['hotel_avg_room_count'] = train_calcs.groupby('prop_id')['srch_room_count'].transform('mean') 
train_calcs['bought'] = train_calcs.groupby('prop_id')['booking_bool'].transform('sum') 
train_calcs = train_calcs[['prop_id','hotel_avg_children', 'hotel_avg_adult', 'hotel_avg_stay','hotel_avg_room_count','bought']]

#pd.crosstab(train.hotel_click_prob, train.prop_id, dropna=False)
train_calcs = train_calcs.drop_duplicates(subset=['prop_id'])

train['hotel_buy_prob'] = train.groupby('prop_id')['booking_bool'].transform('sum') / train.groupby('prop_id')['booking_bool'].transform('count')
train['hotel_click_prob'] = train.groupby('prop_id')['click_bool'].transform('sum') / train.groupby('prop_id')['click_bool'].transform('count')
train['hotel_qrtr_buy_prob'] = train.groupby(['qrtr','prop_id'])['booking_bool'].transform('sum') / train.groupby(['qrtr','prop_id'])['booking_bool'].transform('count')
train['hotel_qrtr_click_prob'] = train.groupby(['qrtr','prop_id'])['click_bool'].transform('sum') / train.groupby(['qrtr','prop_id'])['click_bool'].transform('count')
train['hotel_srch_sesn_buy_prob'] = train.groupby(['seasonality','prop_id'])['booking_bool'].transform('sum') / train.groupby(['seasonality','prop_id'])['booking_bool'].transform('count')
train['hotel_srch_sesn_click_prob'] = train.groupby(['seasonality','prop_id'])['click_bool'].transform('sum') / train.groupby(['seasonality','prop_id'])['click_bool'].transform('count')


hotel_click_stats = train[['prop_id', 'hotel_buy_prob','hotel_click_prob', 'hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']]

<ipython-input-12-6a465437114b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_calcs['hotel_avg_children'] = train_calcs.groupby('prop_id')['srch_children_count'].transform('mean')
<ipython-input-12-6a465437114b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_calcs['hotel_avg_adult'] = train_calcs.groupby('prop_id')['srch_adults_count'].transform('mean')
<ipython-input-12-6a465437114b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [13]:
## balancing the dataset - at first balanced it 50:50 but now balancing it 33:66
df = train[train.outcome == 1]
df1 = train[train.outcome == 0]
df1 = df1.sample(n=len(df)*2)

frames = [df, df1]

train_reduced = pd.concat(frames)

In [14]:
### merging hotel specific cals to main dataset 
train_reduced = train_reduced.merge(train_calcs, left_on=['prop_id'], right_on=['prop_id'], how='left')
train_reduced['hotel_avg_children'] = train_reduced['hotel_avg_children'].replace(np.nan, .367)
train_reduced['hotel_avg_adult'] = train_reduced['hotel_avg_adult'].replace(np.nan, 1.95)
train_reduced['hotel_avg_stay'] = train_reduced['hotel_avg_stay'].replace(np.nan, 2.09)
train_reduced['hotel_avg_room_count'] = train_reduced['hotel_avg_room_count'].replace(np.nan, 1.13)


In [15]:
train_reduced # 64 columns

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,hotel_click_prob,hotel_qrtr_buy_prob,hotel_qrtr_click_prob,hotel_srch_sesn_buy_prob,hotel_srch_sesn_click_prob,hotel_avg_children,hotel_avg_adult,hotel_avg_stay,hotel_avg_room_count,bought
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,68914,2.0,3.0,...,0.034305,0.032258,0.046083,0.038278,0.047847,0.176471,2.058824,1.235294,1.176471,17.0
1,4,2012-12-31 08:59:22,5,219,NaN,NaN,219,139893,2.0,3.0,...,0.033520,0.000000,0.035088,0.027523,0.036697,0.285714,2.142857,1.285714,1.285714,7.0
2,6,2013-06-05 12:27:51,14,100,NaN,NaN,100,104251,3.0,4.0,...,0.045455,0.071429,0.071429,0.090909,0.090909,0.000000,2.000000,1.000000,1.000000,1.0
3,8,2013-03-20 17:50:44,5,219,NaN,NaN,219,27669,3.0,3.5,...,0.035897,0.049383,0.061728,0.040541,0.040541,0.166667,1.666667,1.833333,1.000000,6.0
4,11,2013-02-25 08:39:33,5,219,NaN,NaN,219,20499,2.0,3.5,...,0.030612,0.000000,0.026316,0.026316,0.052632,0.000000,2.000000,1.000000,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665083,243383,2013-02-15 01:03:48,5,219,NaN,NaN,219,130755,2.0,2.5,...,0.017544,0.000000,0.000000,0.071429,0.071429,1.000000,1.000000,2.000000,1.000000,1.0
665084,259550,2013-02-16 20:19:50,15,55,NaN,NaN,100,94591,2.0,3.0,...,0.063158,0.000000,0.000000,0.115385,0.115385,1.000000,2.666667,1.166667,1.333333,6.0
665085,96839,2013-06-04 09:41:04,32,220,NaN,NaN,219,6623,3.0,4.5,...,0.026596,0.019608,0.019608,0.000000,0.000000,0.000000,2.000000,2.600000,1.200000,5.0
665086,253971,2013-01-11 17:13:12,5,219,NaN,NaN,88,36737,4.0,3.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.367000,1.950000,2.090000,1.130000,NaN


In [16]:
#dropped price bin
train_reduced = train_reduced.drop(['price_bin'], axis = 1)

In [17]:
#checking %% of nulls in the dataset 
train_reduced.isnull().mean() * 100

srch_id                         0.000000
date_time                       0.000000
site_id                         0.000000
visitor_location_country_id     0.000000
visitor_hist_starrating        94.945331
                                 ...    
hotel_avg_children              0.000000
hotel_avg_adult                 0.000000
hotel_avg_stay                  0.000000
hotel_avg_room_count            0.000000
bought                         20.419102
Length: 63, dtype: float64

In [18]:
##droppping columns not needed 

# to_drop = ['srch_id','site_id', 'date_time', 'visitor_location_country_id', 'prop_country_id', 'prop_id', 'prop_brand_bool', 'orig_destination_distance', 'click_bool', 'booking_bool', 'srch_destination_id', 'position', 'prop_location_score2'
#           ,'gross_bookings_usd', 'price_usd', 'max_price','min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star', 'bought', 'weekday', 'week_of_year', 'hour', 'time_epoch'] 

to_drop = ['site_id', 'date_time', 'visitor_location_country_id', 'prop_country_id', 'prop_brand_bool', 'orig_destination_distance', 'click_bool', 'booking_bool', 'srch_destination_id', 'position', 'prop_location_score2'
          ,'gross_bookings_usd', 'price_usd', 'max_price','min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star', 'bought', 'weekday', 'week_of_year', 'hour', 'time_epoch'] 


#'room_count_grt_1_flag',
train1 = train_reduced.drop(to_drop,  axis=1)

In [19]:
train1 # 40 columns

,srch_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_id,prop_starrating,prop_review_score,prop_location_score1,prop_log_historical_price,promotion_flag,srch_length_of_stay,...,hotel_buy_prob,hotel_click_prob,hotel_qrtr_buy_prob,hotel_qrtr_click_prob,hotel_srch_sesn_buy_prob,hotel_srch_sesn_click_prob,hotel_avg_children,hotel_avg_adult,hotel_avg_stay,hotel_avg_room_count
0,1,NaN,NaN,68914,2.0,3.0,2.20,4.44,0,1,...,0.029160,0.034305,0.032258,0.046083,0.038278,0.047847,0.176471,2.058824,1.235294,1.176471
1,4,NaN,NaN,139893,2.0,3.0,1.61,5.25,0,2,...,0.019553,0.033520,0.000000,0.035088,0.027523,0.036697,0.285714,2.142857,1.285714,1.285714
2,6,NaN,NaN,104251,3.0,4.0,1.95,0.00,0,1,...,0.045455,0.045455,0.071429,0.071429,0.090909,0.090909,0.000000,2.000000,1.000000,1.000000
3,8,NaN,NaN,27669,3.0,3.5,3.00,4.62,0,1,...,0.030769,0.035897,0.049383,0.061728,0.040541,0.040541,0.166667,1.666667,1.833333,1.000000
4,11,NaN,NaN,20499,2.0,3.5,2.20,4.11,0,1,...,0.010204,0.030612,0.000000,0.026316,0.026316,0.052632,0.000000,2.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665083,243383,NaN,NaN,130755,2.0,2.5,0.00,4.67,0,1,...,0.017544,0.017544,0.000000,0.000000,0.071429,0.071429,1.000000,1.000000,2.000000,1.000000
665084,259550,NaN,NaN,94591,2.0,3.0,2.64,4.57,0,1,...,0.063158,0.063158,0.000000,0.000000,0.115385,0.115385,1.000000,2.666667,1.166667,1.333333
665085,96839,NaN,NaN,6623,3.0,4.5,4.36,0.00,0,10,...,0.013298,0.026596,0.019608,0.019608,0.000000,0.000000,0.000000,2.000000,2.600000,1.200000
665086,253971,NaN,NaN,36737,4.0,3.0,1.39,5.71,0,18,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.367000,1.950000,2.090000,1.130000


In [20]:
##changing nan or null values to 0
train1.replace([np.inf, -np.inf], 0, inplace=True)

train1=train1.dropna(axis=1,how="any")
#msn.bar(train1,figsize=(18,3), color='red')

In [21]:
##ordering columns and selecting predictors + outcome
#train1 = train1[['prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 'hotel_buy_prob','diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score','price_review_value', 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay','hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob','outcome']]
train1 = train1[['srch_id', 'prop_id', 'grade', 'prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 'hotel_buy_prob','diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score','price_review_value', 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay','hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']]

In [22]:
train1.columns # 31 columns

Index(['srch_id', 'prop_id', 'grade', 'prop_starrating',
       'prop_location_score1', 'prop_log_historical_price', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'hotel_click_prob', 'hotel_buy_prob', 'diff_max_price',
       'diff_min_price', 'diff_mean_price', 'hotel_show_prob',
       'prop_review_score', 'price_review_value', 'hotel_avg_children',
       'hotel_avg_adult', 'hotel_avg_stay', 'hotel_avg_room_count',
       'random_bool', 'qrtr', 'hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob',
       'hotel_srch_sesn_buy_prob', 'hotel_srch_sesn_click_prob'],
      dtype='object')

In [23]:
def create_features_column(row, columns):
    merged = []
    for c in columns:
        try:
            merged.append(row[c])
        except:
            import pdb; pdb.set_trace()
            
    return merged        
        
    
columns = ['prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 'hotel_buy_prob','diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score','price_review_value', 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay','hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']

train1['features'] = train1.apply(lambda row: create_features_column(row, columns), axis=1)

In [25]:
columns_drop = ['prop_starrating', 
 'prop_location_score1',
 'prop_log_historical_price',
 'promotion_flag','srch_length_of_stay',
 'srch_booking_window','srch_adults_count','srch_children_count',
 'srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 
 'hotel_buy_prob','diff_max_price','diff_min_price','diff_mean_price',
 'hotel_show_prob', 'prop_review_score','price_review_value', 
 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay',
 'hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 
 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']



train1 = train1.drop(columns_drop,  axis=1)

In [ ]:
#splitting dataset 
# X = train1.iloc[:, :-1].values
# y = train1.iloc[:, -1].values

In [ ]:
# # Splitting the dataset into the Training set and Test set
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [26]:
def compute_lambdas(lambdas_per_query):
    lambdas_per_query = lambdas_per_query.sort_values(['srch_id', 'last_prediction'], ascending=[True, False], kind='stable')
    lambdas_per_query['display_rank'] = lambdas_per_query.groupby('srch_id').cumcount()

    #TBD - How do generalize this to any metric?
    lambdas_per_query['discount'] = 1 / np.log2(2 + lambdas_per_query['display_rank'])
    lambdas_per_query['gain'] = (2**lambdas_per_query['grade'] - 1)

    # swaps dataframe holds each pair-wise swap computed (shrink columns for memory?)   
    # Optimization of swaps = lambdas_per_query.merge(lambdas_per_query, on='qid', how='outer')
    # to limit to just needed columns
    to_swap = lambdas_per_query[['srch_id', 'display_rank', 'grade', 'last_prediction', 'discount', 'gain']]
    #to_swap = lambdas_per_query
    swaps = to_swap.merge(to_swap, on='srch_id', how='outer')

    # delta - delta in DCG due to swap
    swaps['delta'] = np.abs((swaps['discount_x'] - swaps['discount_y']) * (swaps['gain_x'] - swaps['gain_y']))
    
    # rho - based on current model prediction delta
    swaps['rho'] = 1 / (1 + np.exp(swaps['last_prediction_x'] - swaps['last_prediction_y']))
    
    # If you want to be pure gradient boosting, weight reweights each models prediction
    # I haven't found this to matter in practice
    swaps['weight'] = swaps['rho'] * (1.0 - swaps['rho']) * swaps['delta']

    # Compute lambdas (the next model in ensemble's predictors) when grade_x > grade_y
    swaps['lambda'] = 0
    slice_x_better =swaps[swaps['grade_x'] > swaps['grade_y']]
    swaps.loc[swaps['grade_x'] > swaps['grade_y'], 'lambda'] = slice_x_better['delta'] * slice_x_better['rho']
    
    # accumulate lambdas and add back to model
    lambdas_x = swaps.groupby(['srch_id', 'display_rank_x'])['lambda'].sum().rename('lambda')
    lambdas_y = swaps.groupby(['srch_id', 'display_rank_y'])['lambda'].sum().rename('lambda')

    weights_x = swaps.groupby(['srch_id', 'display_rank_x'])['weight'].sum().rename('weight')
    weights_y = swaps.groupby(['srch_id', 'display_rank_y'])['weight'].sum().rename('weight')
    
    weights = weights_x + weights_y
    lambdas = lambdas_x - lambdas_y

    lambdas_per_query = lambdas_per_query.merge(lambdas, 
                                                left_on=['srch_id', 'display_rank'], 
                                                right_on=['srch_id', 'display_rank_x'], 
                                                how='left')
    lambdas_per_query = lambdas_per_query.merge(weights, 
                                                left_on=['srch_id', 'display_rank'], 
                                                right_on=['srch_id', 'display_rank_x'], 
                                                how='left')

    return lambdas_per_query

In [28]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd


ensemble=[]
def lambda_mart_pure(judgments, rounds=20,
                     learning_rate=0.1, max_leaf_nodes=8):

    print(judgments.columns)

    lambdas_per_query = judgments.copy()
    lambdas_per_query['last_prediction'] = 0.0

    for i in range(0, rounds):
        print(f"round {i}")

        # ------------------
        #1. Build pair-wise predictors for this round
        lambdas_per_query = compute_lambdas(lambdas_per_query)

        # ------------------
        #2. Train a regression tree on this round's lambdas
        features = lambdas_per_query['features'].tolist()
        new_tree = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
        new_tree.fit(features, lambdas_per_query['lambda'])    

        # -------------------
        #4. Add to ensemble, recreate last prediction
        ensemble.append(new_tree)
        next_predictions = new_tree.predict(features)
        lambdas_per_query['last_prediction'] += (next_predictions * learning_rate) 
        
        print("*********")
        print(lambdas_per_query.loc[0, ['grade', 'last_prediction']])
        print("*********")
        
        print("Train DCGs")
        lambdas_per_query['discounted_gain'] = lambdas_per_query['gain'] * lambdas_per_query['discount'] 
        dcg = lambdas_per_query[lambdas_per_query['display_rank'] < 10].groupby('srch_id')['discounted_gain'].sum().mean()
        print("mean   ", dcg)
        print("----------")
        
        lambdas_per_query = lambdas_per_query.drop(['lambda', 'weight'], axis=1)
    return lambdas_per_query


lambdas_per_query = lambda_mart_pure(judgments=train1, rounds=100, max_leaf_nodes=20, learning_rate=0.10)

Index(['srch_id', 'prop_id', 'grade', 'features'], dtype='object')
round 0
*********
grade                     5
last_prediction    0.071858
Name: 0, dtype: object
*********
Train DCGs
mean    21.53798196833927
----------
round 1
*********
grade                   5
last_prediction    0.1372
Name: 0, dtype: object
*********
Train DCGs
mean    19.992942294929172
----------
round 2
*********
grade                     0
last_prediction    0.290867
Name: 0, dtype: object
*********
Train DCGs
mean    19.809886746446463
----------
round 3
*********
grade                     0
last_prediction    0.394291
Name: 0, dtype: object
*********
Train DCGs
mean    19.73632841378236
----------
round 4
*********
grade                     0
last_prediction    0.419503
Name: 0, dtype: object
*********
Train DCGs
mean    19.70673174692287
----------
round 5
*********
grade                     0
last_prediction    0.433738
Name: 0, dtype: object
*********
Train DCGs
mean    19.69008395472014
----------
round

*********
grade                     0
last_prediction    1.737635
Name: 0, dtype: object
*********
Train DCGs
mean    20.11073550243602
----------
round 53
*********
grade                     0
last_prediction    1.747969
Name: 0, dtype: object
*********
Train DCGs
mean    20.111874716866442
----------
round 54
*********
grade                     0
last_prediction    1.763921
Name: 0, dtype: object
*********
Train DCGs
mean    20.113978119435366
----------
round 55
*********
grade                     0
last_prediction    1.770535
Name: 0, dtype: object
*********
Train DCGs
mean    20.116775060989553
----------
round 56
*********
grade                     0
last_prediction    1.800788
Name: 0, dtype: object
*********
Train DCGs
mean    20.11912818743052
----------
round 57
*********
grade                     0
last_prediction    1.819937
Name: 0, dtype: object
*********
Train DCGs
mean    20.126924905668027
----------
round 58
*********
grade                     0
last_prediction    1.8

In [36]:
len(train1['features'][0])

28

In [29]:
# ----------------------------------- #
###################### pred 
to_pred=pd.read_csv("../test_set_VU_DM.csv")

In [30]:

to_drop= ['comp1_rate',
'comp1_inv',
'comp1_rate_percent_diff',
'comp2_rate',
'comp2_inv',
'comp2_rate_percent_diff',
'comp3_rate',
'comp3_inv',
'comp3_rate_percent_diff',
'comp4_rate',
'comp4_inv',
'comp4_rate_percent_diff',
'comp5_rate',
'comp5_inv',
'comp5_rate_percent_diff',
'comp6_rate',
'comp6_inv',
'comp6_rate_percent_diff',
'comp7_rate',
'comp7_inv',
'comp7_rate_percent_diff',
'comp8_rate',
'comp8_inv',
'comp8_rate_percent_diff'

]


to_pred = to_pred.drop(to_drop,  axis=1)#.sample(frac=0.2, replace=True, random_state=1)
#test_reduced  = test.iloc[: , :N].sample(frac=0.33, replace=True, random_state=1)


In [31]:
# creating same features as test data

to_pred['date_time'] = pd.to_datetime(to_pred['date_time'], errors='coerce')
to_pred["weekday"] = to_pred["date_time"].dt.weekday
to_pred["week_of_year"] = to_pred["date_time"].dt.week
to_pred["month"] = to_pred["date_time"].dt.month
to_pred["hour"] = to_pred["date_time"].dt.hour
to_pred["time_epoch"] = to_pred["date_time"].astype('int64')//1e9
to_pred["time_epoch"] = to_pred["date_time"].astype('int64')//1e9
to_pred.loc[ to_pred['hour'] < 6, 'day_time'] = 1
to_pred.loc[(to_pred['hour'] >=6) & (to_pred['hour'] <= 11), 'day_time'] = 2
to_pred.loc[(to_pred['hour'] >= 12) & (to_pred['hour'] <= 17), 'day_time'] = 3
to_pred.loc[(to_pred['hour'] > 18) , 'day_time'] = 4

##################

to_pred['srch_for_date'] = to_pred['date_time']  + pd.to_timedelta(to_pred['srch_booking_window'], unit='D')
to_pred["srch_for_month"] = to_pred["srch_for_date"].dt.month

#train['srch_for_date'] = train['date_time']  + pd.to_timedelta(train['srch_booking_window'], unit='D')
#train["srch_for_month"] = train["srch_for_date"].dt.month
to_pred.loc[ to_pred['srch_for_month'] <= 3, 'seasonality'] = 1
to_pred.loc[(to_pred['srch_for_month'] > 3) & (to_pred['srch_for_month'] <= 6), 'seasonality'] = 2
to_pred.loc[(to_pred['srch_for_month'] > 6) & (to_pred['srch_for_month'] <= 9), 'seasonality'] = 3
to_pred.loc[(to_pred['srch_for_month'] > 9) , 'seasonality'] = 4

to_pred.loc[ to_pred['month'] <= 3, 'qrtr'] = 1
to_pred.loc[(to_pred['month'] > 3) & (to_pred['month'] <= 6), 'qrtr'] = 2
to_pred.loc[(to_pred['month'] > 6) & (to_pred['month'] <= 9), 'qrtr'] = 3
to_pred.loc[(to_pred['month'] > 9) , 'qrtr'] = 4

####################

to_pred['price_bin'] = pd.qcut(to_pred['price_usd'], q=80, precision=0)
to_pred['mean_price_bin_star'] = to_pred.groupby('price_bin')['prop_starrating'].transform('mean')
to_pred['mean_price_bin_review'] = to_pred.groupby('price_bin')['prop_review_score'].transform('mean')
to_pred['prop_review_score'] = to_pred['prop_review_score'].fillna(0)
to_pred.loc[to_pred.prop_review_score == 0, "prop_review_score"] = to_pred.mean_price_bin_review
to_pred.loc[to_pred.prop_starrating == 0, "prop_starrating"] = to_pred.mean_price_bin_star

##################

to_pred['max_price'] = to_pred.groupby('srch_id')['price_usd'].transform('max') 
to_pred['min_price'] = to_pred.groupby('srch_id')['price_usd'].transform('min') 
to_pred['mean_price'] = to_pred.groupby('srch_id')['price_usd'].transform('mean') 
to_pred['diff_max_price'] =(to_pred['max_price'] -to_pred['price_usd']) / to_pred['price_usd']
to_pred['diff_min_price'] = (to_pred['min_price'] - to_pred['price_usd'])/ to_pred['price_usd']
to_pred['diff_min_price'] = to_pred['diff_min_price'].replace(np.nan, 0)
to_pred['diff_mean_price'] = (to_pred['mean_price'] - to_pred['price_usd']) / to_pred['price_usd']
to_pred['price_review_value'] =  to_pred['prop_review_score'] /to_pred['mean_price_bin_review'] 

#################

to_pred['hotel_show_prob'] =  to_pred.groupby('prop_id')['prop_id'].transform('count') / (to_pred.srch_id.unique()).size

<ipython-input-31-d9c87d54dfbc>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  to_pred["week_of_year"] = to_pred["date_time"].dt.week


In [32]:
### mergeing hotel specific stats from test dataset 

to_pred_reduced = to_pred.merge(train_calcs, left_on=['prop_id'], right_on=['prop_id'], how='left')
hotel_click_stats = hotel_click_stats.drop_duplicates(subset=['prop_id'])
to_pred_reduced = to_pred_reduced.merge(hotel_click_stats, left_on=['prop_id'], right_on=['prop_id'], how='left')

In [33]:

#train_reduced = train_reduced.merge(train_clacs, on='prop_id', how='right', suffixes=('_1', '_2'))
#to_pred_reduced  = pd.concat([to_pred_reduced.set_index('prop_id'),train_calcs.set_index('prop_id')], axis=1, join='outer').reset_index()
#to_pred_reduced2  = pd.concat([to_pred_reduced.set_index('prop_id'),hotel_click_stats.set_index('prop_id')], axis=1, join='outer').reset_index()
to_pred_reduced['hotel_avg_children'] = to_pred_reduced['hotel_avg_children'].replace(np.nan, .367)
to_pred_reduced['hotel_avg_adult'] = to_pred_reduced['hotel_avg_adult'].replace(np.nan, 1.95)
to_pred_reduced['hotel_avg_stay'] = to_pred_reduced['hotel_avg_stay'].replace(np.nan, 2.09)
to_pred_reduced['hotel_avg_room_count'] = to_pred_reduced['hotel_avg_room_count'].replace(np.nan, 1.13)
to_pred_reduced[['hotel_click_prob', 'hotel_buy_prob']] = to_pred_reduced[['hotel_click_prob', 'hotel_buy_prob']].fillna(0)


#train_reduced.to_csv('hotel_calcs2.csv')

In [37]:
###droppping unused columns 

to_drop = ['site_id', 'date_time', 'visitor_location_country_id', 'prop_country_id','price_bin', 'prop_brand_bool', 'orig_destination_distance', 'srch_destination_id', 'prop_location_score2'
          ,'price_usd','price_bin', 'max_price', 'srch_query_affinity_score','visitor_hist_starrating', 'visitor_hist_adr_usd', 'min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star', 'bought',  'weekday', 'week_of_year', 'hour', 'time_epoch'] 

to_pred1 = to_pred_reduced.drop(to_drop,  axis=1)

In [38]:
#replacing nulls and nans

to_pred1.replace([np.inf, -np.inf], 0, inplace=True)

to_pred1[['hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob', 'hotel_srch_sesn_click_prob']] = to_pred1[['hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob', 'hotel_srch_sesn_click_prob']].replace(np.nan, 0)

to_pred1=to_pred1.dropna(axis=1,how="any")
#msn.bar(to_pred1,figsize=(18,3), color='red')

In [39]:
##ordering df and selecting predictors
to_pred1 = to_pred1[['srch_id', 'prop_id', 'prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 'hotel_buy_prob','diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score','price_review_value', 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay','hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']]

In [40]:
to_pred1.columns

Index(['srch_id', 'prop_id', 'prop_starrating', 'prop_location_score1',
       'prop_log_historical_price', 'promotion_flag', 'srch_length_of_stay',
       'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool', 'hotel_click_prob',
       'hotel_buy_prob', 'diff_max_price', 'diff_min_price', 'diff_mean_price',
       'hotel_show_prob', 'prop_review_score', 'price_review_value',
       'hotel_avg_children', 'hotel_avg_adult', 'hotel_avg_stay',
       'hotel_avg_room_count', 'random_bool', 'qrtr', 'hotel_qrtr_click_prob',
       'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob',
       'hotel_srch_sesn_click_prob'],
      dtype='object')

In [41]:
def create_features_column(row, columns):
    merged = []
    for c in columns:
        try:
            merged.append(row[c])
        except:
            import pdb; pdb.set_trace()
            
    return merged        
        
columns = ['prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag',
           'srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count',
           'srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 'hotel_buy_prob',
           'diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score',
           'price_review_value', 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay',
           'hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 
           'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']

to_pred1['features'] = to_pred1.apply(lambda row: create_features_column(row, columns), axis=1)

In [42]:
columns_drop = ['prop_starrating', 
 'prop_location_score1',
 'prop_log_historical_price',
 'promotion_flag','srch_length_of_stay',
 'srch_booking_window','srch_adults_count','srch_children_count',
 'srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 
 'hotel_buy_prob','diff_max_price','diff_min_price','diff_mean_price',
 'hotel_show_prob', 'prop_review_score','price_review_value', 
 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay',
 'hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 
 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']



to_pred1 = to_pred1.drop(columns_drop,  axis=1)

In [44]:
len(to_pred1['features'][0])

28

In [50]:

# ------------------
#1. Extract features from test dataframe
features = to_pred1['features'].tolist()

#2. For each estimator in the ensemble, evaluate test set
i = 0
predictions = [0] * len(to_pred1)
for tree in ensemble:
    print(i)
    new_predictions = tree.predict(features)
    predictions += new_predictions
    
    #import pdb; pdb.set_trace()
    i += 1
    print("----------")
    
predictions = predictions / 50

0
----------
1
----------
2
----------
3
----------
4
----------
5
----------
6
----------
7
----------
8
----------
9
----------
10
----------
11
----------
12
----------
13
----------
14
----------
15
----------
16
----------
17
----------
18
----------
19
----------
20
----------
21
----------
22
----------
23
----------
24
----------
25
----------
26
----------
27
----------
28
----------
29
----------
30
----------
31
----------
32
----------
33
----------
34
----------
35
----------
36
----------
37
----------
38
----------
39
----------
40
----------
41
----------
42
----------
43
----------
44
----------
45
----------
46
----------
47
----------
48
----------
49
----------
50
----------
51
----------
52
----------
53
----------
54
----------
55
----------
56
----------
57
----------
58
----------
59
----------
60
----------
61
----------
62
----------
63
----------
64
----------
65
----------
66
----------
67
----------
68
----------
69
----------
70
----------
71
----------
72

In [51]:
predictions = predictions / 2

In [52]:
predictions

array([ 0.11433572,  0.16794247,  0.04297708, ..., -0.29508765,
       -0.1619423 , -0.29508765])

In [54]:
to_pred1['pred'] = predictions

In [187]:
to_pred2 = to_pred1[['srch_id', 'prop_id', 'pred']]

In [188]:
to_pred2.head(50)

,srch_id,prop_id,pred
0,1,3180,0.114336
1,1,5543,0.167942
2,1,14142,0.042977
3,1,22393,-0.316588
4,1,24194,0.067957
5,1,28181,0.187035
6,1,34263,-0.290830
7,1,37567,-0.282103
8,1,50162,-0.316588
9,1,54937,0.082648


In [189]:

to_pred2 = to_pred2.groupby('srch_id').apply(lambda x: x.sort_values('pred', ascending = False))


In [190]:
to_pred2.head(20)

srch_id  prop_id      pred
srch_id                               
1       5         1    28181  0.187035
        1         1     5543  0.167942
        23        1    99484  0.151765
        0         1     3180  0.114336
        9         1    54937  0.082648
        12        1    61934  0.069941
        4         1    24194  0.067957
        22        1    95031  0.060381
        2         1    14142  0.042977
        25        1   128085 -0.027006
        17        1    78599 -0.043864
        10        1    56050 -0.050514
        28        1   139162 -0.269009
        16        1    74045 -0.278832
        7         1    37567 -0.282103
        6         1    34263 -0.290830
        21        1    94729 -0.295305
        20        1    90385 -0.298868
        13        1    63894 -0.308797
        11        1    61632 -0.316588

In [191]:
to_pred2 = to_pred2.drop(['pred'], axis = 1)
to_pred2.to_csv('submission.csv', index=False)



In [194]:
to_pred2

srch_id  prop_id
srch_id                          
1       5              1    28181
        1              1     5543
        23             1    99484
        0              1     3180
        9              1    54937
...                  ...      ...
332787  4959177   332787    29018
        4959178   332787    32019
        4959179   332787    33959
        4959180   332787    35240
        4959182   332787    99509

[4959183 rows x 2 columns]